# 🎲 Entropy Fundamentals & Huffman Coding

This notebook provides hands-on exploration of:
1. Information content and surprise
2. Entropy computation and visualization
3. Huffman coding implementation
4. Cross-entropy and its relationship to ML loss

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import heapq
from dataclasses import dataclass, field
from typing import Optional, Dict

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

## 1. Information Content: Measuring Surprise

$$I(x) = -\log_2 P(x) = \log_2 \frac{1}{P(x)}$$

In [ ]:
def information_content(p: float) -> float:
    """Calculate information content in bits."""
    if p <= 0:
        return float('inf')
    if p >= 1:
        return 0
    return -np.log2(p)

# Visualize information content vs probability
probabilities = np.linspace(0.001, 1, 1000)
information = -np.log2(probabilities)

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(probabilities, information, 'b-', linewidth=2)
ax.set_xlabel('Probability P(x)')
ax.set_ylabel('Information I(x) = -log₂(P(x)) [bits]')
ax.set_title('Information Content: Rare Events are Surprising')

# Add annotations for specific points
examples = [(0.5, 'Fair coin flip'), (0.1, 'Rare event'), (0.01, 'Very rare')]
for p, label in examples:
    info = information_content(p)
    ax.annotate(f'{label}\nP={p}, I={info:.2f} bits', 
                xy=(p, info), xytext=(p+0.15, info+0.5),
                arrowprops=dict(arrowstyle='->', color='red'),
                fontsize=10)
    ax.plot(p, info, 'ro', markersize=8)

ax.set_xlim(0, 1.1)
ax.set_ylim(0, 10)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 2. Entropy: Expected Surprise

$$H(X) = -\sum_x P(x) \log_2 P(x)$$

Let's explore how entropy changes with distribution shape.

In [ ]:
def entropy(probs: np.ndarray) -> float:
    """Calculate entropy of a discrete distribution."""
    probs = np.array(probs)
    probs = probs[probs > 0]  # Avoid log(0)
    return -np.sum(probs * np.log2(probs))

# Binary entropy function H(p) for a coin with P(heads) = p
def binary_entropy(p: float) -> float:
    if p <= 0 or p >= 1:
        return 0
    return -p * np.log2(p) - (1-p) * np.log2(1-p)

# Plot binary entropy
p_values = np.linspace(0.001, 0.999, 1000)
h_values = [binary_entropy(p) for p in p_values]

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(p_values, h_values, 'b-', linewidth=2)
ax.set_xlabel('P(heads)')
ax.set_ylabel('Entropy H(X) [bits]')
ax.set_title('Binary Entropy: Maximum Uncertainty at p=0.5')

# Mark maximum
ax.axvline(x=0.5, color='r', linestyle='--', alpha=0.5)
ax.plot(0.5, 1.0, 'ro', markersize=10)
ax.annotate('Maximum entropy\nH=1 bit at p=0.5', xy=(0.5, 1), xytext=(0.65, 0.85),
            arrowprops=dict(arrowstyle='->', color='red'), fontsize=11)

# Add interpretation regions
ax.fill_between(p_values[:250], h_values[:250], alpha=0.3, color='green', label='Predictable (biased towards tails)')
ax.fill_between(p_values[750:], h_values[750:], alpha=0.3, color='green', label='Predictable (biased towards heads)')
ax.fill_between(p_values[400:600], h_values[400:600], alpha=0.3, color='red', label='High uncertainty')

ax.set_xlim(0, 1)
ax.set_ylim(0, 1.1)
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Compare entropy of different distributions
distributions = {
    'Uniform (4 outcomes)': [0.25, 0.25, 0.25, 0.25],
    'Slightly biased': [0.4, 0.3, 0.2, 0.1],
    'Very biased': [0.7, 0.1, 0.1, 0.1],
    'Almost deterministic': [0.97, 0.01, 0.01, 0.01]
}

fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.flatten()

for ax, (name, probs) in zip(axes, distributions.items()):
    h = entropy(probs)
    colors = plt.cm.Blues(np.array(probs) / max(probs))
    bars = ax.bar(range(len(probs)), probs, color=colors, edgecolor='black')
    ax.set_title(f'{name}\nH = {h:.3f} bits', fontsize=12)
    ax.set_xlabel('Outcome')
    ax.set_ylabel('Probability')
    ax.set_ylim(0, 1)
    ax.set_xticks(range(len(probs)))
    
    # Add probability labels on bars
    for i, (bar, p) in enumerate(zip(bars, probs)):
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02, 
                f'{p:.2f}', ha='center', fontsize=10)

plt.suptitle('Entropy Decreases as Distribution Becomes More Predictable', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

print("\nEntropy comparison:")
print(f"Maximum possible (uniform over 4): {np.log2(4):.3f} bits")
for name, probs in distributions.items():
    print(f"{name}: {entropy(probs):.3f} bits")

## 3. Huffman Coding: Achieving Entropy Bounds

Huffman coding creates an optimal prefix-free code that approaches the entropy bound.

In [ ]:
@dataclass(order=True)
class HuffmanNode:
    """Node in a Huffman tree."""
    prob: float
    symbol: Optional[str] = field(compare=False, default=None)
    left: Optional['HuffmanNode'] = field(compare=False, default=None)
    right: Optional['HuffmanNode'] = field(compare=False, default=None)
    
    def is_leaf(self) -> bool:
        return self.symbol is not None


def build_huffman_tree(symbol_probs: Dict[str, float]) -> HuffmanNode:
    """Build Huffman tree from symbol probabilities."""
    # Create leaf nodes
    heap = [HuffmanNode(prob=p, symbol=s) for s, p in symbol_probs.items()]
    heapq.heapify(heap)
    
    # Build tree
    while len(heap) > 1:
        left = heapq.heappop(heap)
        right = heapq.heappop(heap)
        merged = HuffmanNode(
            prob=left.prob + right.prob,
            left=left,
            right=right
        )
        heapq.heappush(heap, merged)
    
    return heap[0]


def get_huffman_codes(root: HuffmanNode, prefix: str = '') -> Dict[str, str]:
    """Extract codes from Huffman tree."""
    if root.is_leaf():
        return {root.symbol: prefix or '0'}  # Handle single symbol case
    
    codes = {}
    if root.left:
        codes.update(get_huffman_codes(root.left, prefix + '0'))
    if root.right:
        codes.update(get_huffman_codes(root.right, prefix + '1'))
    return codes


def average_code_length(codes: Dict[str, str], probs: Dict[str, float]) -> float:
    """Calculate average code length."""
    return sum(len(codes[s]) * p for s, p in probs.items())

In [ ]:
# Example: Weather forecasts
weather_probs = {
    '☀️ Sunny': 0.5,
    '☁️ Cloudy': 0.25,
    '🌧️ Rainy': 0.125,
    '❄️ Snowy': 0.125
}

# Build Huffman tree and get codes
tree = build_huffman_tree(weather_probs)
codes = get_huffman_codes(tree)

# Calculate metrics
h = entropy(list(weather_probs.values()))
avg_len = average_code_length(codes, weather_probs)

print("Weather Forecast Huffman Coding")
print("=" * 50)
print(f"{'Symbol':<15} {'Probability':<12} {'Code':<10} {'Code Length'}")
print("-" * 50)
for symbol, prob in sorted(weather_probs.items(), key=lambda x: -x[1]):
    code = codes[symbol]
    print(f"{symbol:<15} {prob:<12.3f} {code:<10} {len(code)}")
print("-" * 50)
print(f"\nEntropy H(X): {h:.4f} bits")
print(f"Average code length: {avg_len:.4f} bits")
print(f"Efficiency: {h/avg_len*100:.2f}%")

In [ ]:
# Visualize Huffman coding efficiency across different distributions
def analyze_huffman_efficiency(n_symbols: int, n_trials: int = 100):
    """Compare Huffman code length to entropy for random distributions."""
    entropies = []
    avg_lengths = []
    
    for _ in range(n_trials):
        # Generate random probability distribution
        raw = np.random.exponential(1, n_symbols)
        probs = raw / raw.sum()
        
        symbol_probs = {str(i): p for i, p in enumerate(probs)}
        tree = build_huffman_tree(symbol_probs)
        codes = get_huffman_codes(tree)
        
        entropies.append(entropy(probs))
        avg_lengths.append(average_code_length(codes, symbol_probs))
    
    return np.array(entropies), np.array(avg_lengths)

# Run analysis
entropies, avg_lengths = analyze_huffman_efficiency(8, 500)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Scatter plot
ax = axes[0]
ax.scatter(entropies, avg_lengths, alpha=0.5, s=30)
ax.plot([0, 3.5], [0, 3.5], 'r--', linewidth=2, label='H(X) = Avg Length (ideal)')
ax.plot([0, 3.5], [1, 4.5], 'g--', linewidth=2, label='H(X) + 1 (upper bound)')
ax.set_xlabel('Entropy H(X) [bits]')
ax.set_ylabel('Average Huffman Code Length [bits]')
ax.set_title('Huffman Coding Achieves Near-Entropy Code Length')
ax.legend()
ax.set_xlim(0, 3.5)
ax.set_ylim(0, 4.5)
ax.grid(True, alpha=0.3)

# Efficiency histogram
ax = axes[1]
overhead = avg_lengths - entropies
ax.hist(overhead, bins=30, edgecolor='black', alpha=0.7)
ax.axvline(x=0, color='r', linestyle='--', linewidth=2, label='Perfect efficiency')
ax.axvline(x=overhead.mean(), color='g', linestyle='-', linewidth=2, label=f'Mean overhead: {overhead.mean():.3f}')
ax.set_xlabel('Overhead: Avg Code Length - Entropy [bits]')
ax.set_ylabel('Count')
ax.set_title('Huffman Coding Overhead Distribution')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nHuffman coding overhead statistics:")
print(f"Mean overhead: {overhead.mean():.4f} bits")
print(f"Max overhead: {overhead.max():.4f} bits")
print(f"Shannon's theorem guarantees overhead < 1 bit")

## 4. Cross-Entropy: Using the Wrong Model

$$H(P, Q) = -\sum_x P(x) \log_2 Q(x)$$

Cross-entropy measures the cost of encoding data from $P$ using a code designed for $Q$.

In [ ]:
def cross_entropy(p: np.ndarray, q: np.ndarray) -> float:
    """Calculate cross-entropy H(P, Q)."""
    p, q = np.array(p), np.array(q)
    # Add small epsilon to avoid log(0)
    q = np.clip(q, 1e-10, 1)
    return -np.sum(p * np.log2(q))

def kl_divergence(p: np.ndarray, q: np.ndarray) -> float:
    """Calculate KL divergence D_KL(P || Q)."""
    return cross_entropy(p, q) - entropy(p)

# True distribution (what nature generates)
p_true = np.array([0.5, 0.3, 0.15, 0.05])

# Model distributions (what we predict)
models = {
    'Perfect model (Q = P)': p_true.copy(),
    'Uniform model': np.array([0.25, 0.25, 0.25, 0.25]),
    'Overconfident wrong': np.array([0.1, 0.1, 0.1, 0.7]),
    'Close but not perfect': np.array([0.45, 0.35, 0.12, 0.08])
}

print("Cross-Entropy Analysis")
print("=" * 60)
print(f"True distribution P: {p_true}")
print(f"Entropy H(P): {entropy(p_true):.4f} bits")
print("\n" + "-" * 60)

results = []
for name, q in models.items():
    h_pq = cross_entropy(p_true, q)
    kl = kl_divergence(p_true, q)
    results.append((name, h_pq, kl))
    print(f"\n{name}")
    print(f"  Q: {q}")
    print(f"  Cross-entropy H(P,Q): {h_pq:.4f} bits")
    print(f"  KL divergence D_KL(P||Q): {kl:.4f} bits")
    print(f"  Extra bits wasted: {kl:.4f}")

In [ ]:
# Visualize cross-entropy decomposition
fig, ax = plt.subplots(figsize=(12, 6))

names = [r[0] for r in results]
cross_entropies = [r[1] for r in results]
kl_divs = [r[2] for r in results]
h_p = entropy(p_true)

x = np.arange(len(names))
width = 0.6

# Stacked bar chart
bars1 = ax.bar(x, [h_p]*len(names), width, label=f'Entropy H(P) = {h_p:.3f}', color='steelblue')
bars2 = ax.bar(x, kl_divs, width, bottom=[h_p]*len(names), label='KL Divergence (waste)', color='coral')

ax.set_xlabel('Model')
ax.set_ylabel('Bits')
ax.set_title('Cross-Entropy = Entropy + KL Divergence\n(Minimum achievable + Waste from wrong model)')
ax.set_xticks(x)
ax.set_xticklabels(names, rotation=15, ha='right')
ax.legend()

# Add value labels
for i, (ce, kl) in enumerate(zip(cross_entropies, kl_divs)):
    ax.text(i, ce + 0.05, f'H(P,Q)={ce:.2f}', ha='center', fontsize=10)

ax.set_ylim(0, max(cross_entropies) * 1.2)
ax.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

## 5. ML Loss as Cross-Entropy: A Classification Example

In [ ]:
def softmax(logits: np.ndarray) -> np.ndarray:
    """Compute softmax probabilities."""
    exp_logits = np.exp(logits - np.max(logits))  # Numerical stability
    return exp_logits / exp_logits.sum()

def cross_entropy_loss(y_true: int, y_pred: np.ndarray) -> float:
    """Cross-entropy loss for classification."""
    return -np.log(y_pred[y_true] + 1e-10)

# Simulate a classifier's predictions during training
n_classes = 4
true_label = 0  # The correct class

# Simulate logits at different training stages
training_stages = {
    'Random init': np.random.randn(n_classes),
    'Early training': np.array([1.0, 0.5, 0.3, 0.2]),
    'Mid training': np.array([2.0, 0.5, 0.3, 0.2]),
    'Late training': np.array([4.0, 0.5, 0.3, 0.2]),
    'Converged': np.array([8.0, 0.5, 0.3, 0.2])
}

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left plot: probability distributions
ax = axes[0]
x = np.arange(n_classes)
width = 0.15

colors = plt.cm.viridis(np.linspace(0, 1, len(training_stages)))
losses = []

for i, (stage, logits) in enumerate(training_stages.items()):
    probs = softmax(logits)
    loss = cross_entropy_loss(true_label, probs)
    losses.append(loss)
    ax.bar(x + i*width, probs, width, label=f'{stage}', color=colors[i], alpha=0.8)

ax.axhline(y=1/n_classes, color='red', linestyle='--', label='Uniform (random guess)')
ax.set_xlabel('Class')
ax.set_ylabel('Predicted Probability')
ax.set_title('Classifier Confidence Over Training\n(True class = 0)')
ax.set_xticks(x + width*2)
ax.set_xticklabels(['Class 0\n(TRUE)', 'Class 1', 'Class 2', 'Class 3'])
ax.legend(loc='upper right')
ax.set_ylim(0, 1.1)

# Right plot: loss curve
ax = axes[1]
ax.plot(range(len(losses)), losses, 'bo-', linewidth=2, markersize=10)
ax.set_xlabel('Training Stage')
ax.set_ylabel('Cross-Entropy Loss')
ax.set_title('Loss Decreases as Model Improves')
ax.set_xticks(range(len(losses)))
ax.set_xticklabels(list(training_stages.keys()), rotation=15, ha='right')

# Add annotations
for i, (stage, loss) in enumerate(zip(training_stages.keys(), losses)):
    ax.annotate(f'{loss:.2f} bits', (i, loss), textcoords="offset points", 
                xytext=(0, 10), ha='center')

ax.axhline(y=0, color='green', linestyle='--', label='Perfect prediction (0 bits)')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n🔑 Key Insight:")
print("Cross-entropy loss = bits needed to encode the true label using our model's distribution")
print("Perfect model → 0 bits (we're certain of the right answer)")
print("Uniform model → log₂(n_classes) bits (maximum uncertainty)")

## 6. Real Text Example: Compression = Understanding

In [ ]:
# Analyze entropy of real text
sample_text = """
Machine learning is a subset of artificial intelligence that enables systems to learn 
and improve from experience without being explicitly programmed. Machine learning 
focuses on the development of computer programs that can access data and use it to 
learn for themselves. The process of learning begins with observations or data, such 
as examples, direct experience, or instruction, in order to look for patterns in data 
and make better decisions in the future based on the examples that we provide.
"""

# Character-level analysis
chars = [c.lower() for c in sample_text if c.isalpha() or c == ' ']
char_counts = Counter(chars)
total_chars = len(chars)
char_probs = {c: count/total_chars for c, count in char_counts.items()}

# Calculate entropy
h_chars = entropy(list(char_probs.values()))

# Build Huffman codes
tree = build_huffman_tree(char_probs)
codes = get_huffman_codes(tree)
avg_len = average_code_length(codes, char_probs)

# Compare to fixed-length encoding
n_symbols = len(char_probs)
fixed_len = np.ceil(np.log2(n_symbols))

print("Text Compression Analysis")
print("=" * 50)
print(f"Text length: {total_chars} characters")
print(f"Unique characters: {n_symbols}")
print(f"\nEntropy: {h_chars:.3f} bits/character")
print(f"Huffman average: {avg_len:.3f} bits/character")
print(f"Fixed-length: {fixed_len:.0f} bits/character")
print(f"\nCompression ratio (vs fixed): {fixed_len/avg_len:.2f}x")
print(f"\nTotal bits needed:")
print(f"  Fixed-length: {total_chars * fixed_len:.0f} bits = {total_chars * fixed_len / 8:.0f} bytes")
print(f"  Huffman: {total_chars * avg_len:.0f} bits = {total_chars * avg_len / 8:.0f} bytes")
print(f"  Savings: {(1 - avg_len/fixed_len)*100:.1f}%")

In [ ]:
# Visualize character frequency distribution
sorted_chars = sorted(char_probs.items(), key=lambda x: -x[1])

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Character frequencies
ax = axes[0]
chars_list = [c if c != ' ' else '␣' for c, _ in sorted_chars]
probs_list = [p for _, p in sorted_chars]
ax.bar(range(len(chars_list)), probs_list, color='steelblue', edgecolor='black')
ax.set_xlabel('Character')
ax.set_ylabel('Probability')
ax.set_title('Character Frequency Distribution in English Text')
ax.set_xticks(range(len(chars_list)))
ax.set_xticklabels(chars_list, fontsize=9)

# Huffman codes
ax = axes[1]
code_lengths = [len(codes[c]) for c, _ in sorted_chars[:15]]
ax.bar(range(len(code_lengths)), code_lengths, color='coral', edgecolor='black')
ax.set_xlabel('Character (sorted by frequency)')
ax.set_ylabel('Huffman Code Length (bits)')
ax.set_title('Huffman Code Lengths\n(Common chars → Short codes)')
ax.set_xticks(range(len(code_lengths)))
ax.set_xticklabels([c if c != ' ' else '␣' for c, _ in sorted_chars[:15]])

plt.tight_layout()
plt.show()

print("\n🔑 Key Insight:")
print("Common characters (space, 'e', 't', 'a') get short codes")
print("Rare characters get longer codes")
print("This is exactly what a good language model learns!")

## Summary

### Key Formulas

| Concept | Formula | Meaning |
|---------|---------|--------|
| Information | $I(x) = -\log_2 P(x)$ | Surprise of event $x$ |
| Entropy | $H(X) = -\sum_x P(x) \log_2 P(x)$ | Expected surprise |
| Cross-Entropy | $H(P,Q) = -\sum_x P(x) \log_2 Q(x)$ | Bits using wrong model |
| KL Divergence | $D_{KL}(P\|Q) = H(P,Q) - H(P)$ | Extra bits from wrong model |

### The ML Connection

1. **Training** = Minimizing cross-entropy = Finding best compression
2. **Better model** = Assigns higher probability to real data = Needs fewer bits
3. **Overfitting** = Memorizing noise = Wasting bits on incompressible patterns
4. **Regularization** = Limiting model complexity = Limiting description length